In [1]:
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from tqdm import tqdm

### Initialize the Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Initializing the Hyperparameters

In [18]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5
load_model = True

### Load Data

In [4]:
train_dataset = datasets.MNIST(root='dataset/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



### Create the CNN Model

In [5]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    super(CNN, self).__init__()
    self.con1 = nn.Conv2d(in_channels=in_channels, 
                          out_channels=8, # cnn features 
                          kernel_size=(3,3), 
                          stride=(1,1), 
                          padding=(1,1))
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.con2 = nn.Conv2d(in_channels=8, 
                          out_channels=16, # cnn features 
                          kernel_size=(3,3), 
                          stride=(1,1), 
                          padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes) # 28 will be divided (halfed) twice becz we'll use pooling twice

  def forward(self, x):
    x = F.relu(self.con1(x))
    x = self.pool(x)
    x = F.relu(self.con2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0],-1)
    x = self.fc1(x)

    return x


### Loading & Saving the model

In [15]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
  print("=> Saving Checkpoint")
  torch.save(state, filename)

In [19]:
def load_checkpoint(checkpoint):
  print("=> Loading Checkpoint")
  model.load_state_dict(checkpoint["state_dict"])
  optimizer.load_state_dict(checkpoint["optimizer"])

### Initialize the Model

In [6]:
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

### Loss and Optimizer

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Remember to give model parameters here

=> Loading Checkpoint


In [26]:
# Loading the model from checkpoint
if load_model:
  load_checkpoint(torch.load("my_checkpoint.pth.tar"))

=> Loading Checkpoint


### Training the Model

In [27]:
for epoch in tqdm(range(num_epochs)):
  losses = []

  # Saving the state of model and optimizer
  if epoch % 3 == 0:
    checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(checkpoint)

  for ind, (data, target) in enumerate(train_loader):
    data = data.to(device=device)
    target = target.to(device=device)
    
    # forward
    scores = model(data)
    loss = criterion(scores, target)
    losses.append(loss)

    # backward
    optimizer.zero_grad() # set all the gradients to 0 for each batch
    loss.backward()

    # gradient descent or adam step - updates the parameters
    optimizer.step()

  mean_loss = sum(losses)/len(losses)

  print(f"\nLoss at Epoch {epoch+1} is {mean_loss}")

  0%|          | 0/5 [00:00<?, ?it/s]

=> Saving Checkpoint


 20%|██        | 1/5 [00:17<01:10, 17.72s/it]


Loss at Epoch 1 is 0.018298428505659103


 40%|████      | 2/5 [00:35<00:53, 17.77s/it]

...


Loss at Epoch 3 is 0.016075368970632553
=> Saving Checkpoint


 80%|████████  | 4/5 [01:13<00:18, 18.50s/it]


Loss at Epoch 4 is 0.01504670362919569


100%|██████████| 5/5 [01:30<00:00, 18.11s/it]


Loss at Epoch 5 is 0.01383102498948574


### Testing the Model

In [28]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval() # Let the model know that this is evaluation mode

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      
      # Which class has the max value
      _, pred = scores.max(dim=1)
      num_correct += (pred == y).sum()
      num_samples += pred.size(0) # 64

    print(f"Got {num_correct} / {num_samples} with accuracy {(float(num_correct)/float(num_samples)*100):.2f}%")

In [29]:
check_accuracy(test_loader, model)

Got 9877 / 10000 with accuracy 98.77%
